In [99]:
import torch
import torch.nn as nn

In [39]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.relu(self.conv(x))

In [50]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out1, out_reduce3, out3, out_reduce5, out5, pool_proj):
        super().__init__()
        
                    
        self.branch1 = ConvBlock(in_channels, out1, 1)
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, out_reduce3, kernel_size=1),
            ConvBlock(out_reduce3, out3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, out_reduce5, kernel_size=1),
            ConvBlock(out_reduce5, out5, kernel_size=5, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, pool_proj, kernel_size=1)
        )
        
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)

In [96]:
class GoogLeNet(nn.Module):
    def __init__(self, in_channels, out_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64, 7, stride=2, padding=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.reduce_conv2 = ConvBlock(64, 64, 1, stride=1, padding=0)
        self.conv2 = ConvBlock(64, 192, 3, stride=1, padding=1)
        self.max_pool2 =  nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception_3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        
        self.max_pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception_4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception_4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception_4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        
        self.max_pool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception_5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        
        self.avg_pool5 = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout5 = nn.Dropout(p=0.4)
        
        self.flatten = nn.Flatten()
        self.fc6 = nn.Linear(1024, 1000)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool1(x)
        
        x = self.reduce_conv2(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        
        x = self.inception_3a(x)
        x = self.inception_3b(x)
        
        x = self.max_pool3(x)
        
        x = self.inception_4a(x)
        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        x = self.inception_4e(x)
        
        x = self.max_pool4(x)
        
        x = self.inception_5a(x)
        x = self.inception_5b(x)
        
        x = self.avg_pool5(x)
        x = self.dropout5(x)
        
        x = self.flatten(x)
        x = self.fc6(x)
        
        
        return x

In [97]:
example = torch.randn(1, 3, 224, 224)

In [98]:
model = GoogLeNet(3, 1000)
output = model(example)
print(output.shape)

torch.Size([1, 1000])
